## Scanning Laser Ophthalmoscope (SLO) Segmentation

This notebook provides a convolutional neural network (UNet) architecture) which performs vessel segmentation in infra-red SLO images (particularly those captured with the Heidelberg Spectralis).

To run this software, you need to first install the necessary packages into a python environment. A separate file is included with instructions for this.

Next, you will need to put all of your SLO images into the "SLO" folder within this directory. Whatever directory structure you put into the original folder will be reproduced in the output folder.

This first cell will load the necessary packages and functions to run the segmentation

In [ ]:
import torch
from torch.nn.functional import softmax
import glob
import cv2
from torchvision.transforms import ToTensor
from PIL import Image
import sys
from tqdm import tqdm
from os import makedirs, path
from numpy import uint8, max as Max, min as Min
import numpy as np
from skimage import morphology as morph, measure
from os.path import exists

Next, we define the post-processing applied to the output vessel maps which improves the quality of the output images (by removing small false positive regions, removing small gaps etc.).

In [ ]:
# Post-processing of the output images
def process(im, strel = morph.disk(5), hole_size = 10, object_size = 150):
    imc = morph.binary_closing(im, strel)*1.
    im = im/np.max(im)
    im_out = np.copy(im)
    imdiff = np.logical_xor(im, imc)*1.
    labeldiff = measure.label(imdiff)
    rp = measure.regionprops(labeldiff)
    for region in rp:
        if region.area < 10:
            continue
        if region.eccentricity < 0.95:
            continue
        if region.axis_minor_length > 5:
            continue
        im_out[tuple(np.rot90(region.coords, -1))] = 1.
    
    im_out = morph.remove_small_holes(im_out > 0, hole_size)*1.
    im_out = morph.remove_small_objects(im_out > 0, object_size)*1.
    return(im_out)

Next, you must define the image format you would like to segment. This can be done by changing tif to whatever you want in the following cell. If you have more than one format within your dataset, run this notebook separately for each format.

In [ ]:
format = '.tif'

The notebook will now load in the segmentation network.

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = torch.load('unet/model_trained.pth').to(device)

Finally, the system will iterate through all files in the "SLO" folder and save the output vessel maps in the "SLO_Out" folder (the folder will be made if it does not already exist)

In [ ]:
#Get filelist
flist = glob.glob('./SLO/**/*%s'%format, recursive = True)
for f in tqdm(flist):
    #load image
    f_out = f.replace('SLO', 'SLO_out', 1).replace(format, '.png')
    if exists(f_out):
        continue
    im = ToTensor()(cv2.imread(f)[:,:,0]).unsqueeze(0).to(device)
    #segment the image
    imout = softmax(model(im.to(device)), dim = 1).detach().cpu().numpy()[0, 1]
    imout = process(imout > 0.5)
    #make output folder
    fold_out = path.split(f_out)[0]
    if not exists(fold_out):
        makedirs(fold_out, exist_ok = True)
    #save output
    cv2.imwrite(f_out, np.uint8(imout*255))

If the output images have not been generated, or you experience other problems, please let me know via the GitHub page for this project: _(link here)_

This project uses the UNet reported by Ronneberger et al. (2015, doi:10.1007/978-3-319-24574-4_28)

The code for this UNet was taken from https://github.com/milesial/Pytorch-UNet